In [146]:
import nltk
from textblob import TextBlob
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
import json
import pymongo
import os
from pprint import pprint
import nytimes


In [151]:
search_obj.article_search(filename="ph_2014.json",q="prospect heights real estate",begin_date='20140101',end_date='20141231')

In [150]:
search_obj = nytimes.get_article_search_obj ('7ff048af1779e0c12cf7038c3d9476a0:15:56968285')

In [148]:
years = [2014,2013,2012,2011,2010]
q_terms = ['Bay Ridge','Bedford-Stuyvesant','Bensonhurst','Bergen Beach','Boerum Hill','Borough Park','Brighton Beach',
           'Brooklyn Heights','Brooklyn Navy Yard','Brownsville','Bushwick','Canarsie','Carroll Gardens',
           'Clinton Hill','Cobble Hill','Coney Island','Crown Heights',
           'Cypress Hills','Ditmas Park','Downtown Brooklyn','DUMBO','Dyker Heights','East New York',
           'East Williamsburg','Farragut','Flatbush','Flatlands','Fort Greene','Fort Hamilton','Gerritsen Beach',
           'Gowanus','Gravesend','Greenpoint','Greenwood Heights','Kensington',
           'Manhattan Beach','Marine Park','Midwood','Mill Basin','New Lots','New Utrecht',
           'Ocean Parkway','Old Mill Basin','Park Slope','Prospect Heights',
           'Prospect Lefferts Gardens','Prospect Park South','Red Hook','Seagate','Sheepshead Bay',
           'Stuyvesant Heights','Sunset Park','Vinegar Hill','White Sands','Williamsburg','Windsor Terrace']

In [152]:
for q in q_terms:
    for y in years:
#       search_obj.article_search(filename=q+str(y)+".json",q="'"+q+" real estate" "'",news_desk='RealEstate',glocations="New York City",begin_date=str(y)+"0101",end_date=str(y)+"1231")
#         search_obj.article_search(filename="'"+q+str(y)+".json" "'",q="'"+q+" real estate" "'",news_desk='RealEstate',glocations="New York City",begin_date=str(y)+"0101",end_date=str(y)+"1231")     
        search_obj.article_search(filename="'"+q+str(y)+".json" "'",q="'"+q+" real estate" "'",begin_date=str(y)+"0101",end_date=str(y)+"1231")     

In [27]:
try:
    conn=pymongo.MongoClient()
    print "Connected successfully!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e 

Connected successfully!!!


In [5]:
path = '/Users/gfsoileau/Metis/Fletcher/times_data'
files = os.listdir(path)

In [183]:
#ny_times = 
for f in files:
    year = f[-10:-6]
    name_hood = f[1:-10]
    print year, name_hood
    f_path ='/Users/gfsoileau/Metis/Fletcher/times_data/'+f
    with open(f_path) as json_file:
        print '*' *20
        data = json_file.read().replace('\n', '')
        times_dict = json.loads(data)
        count = 0
#         for k, v in times_dict['response']['meta'].items():
#             dic[k]=v 
        for i in times_dict['response']['docs']:
            dic = {}
            if i['news_desk'] == 'RealEstate' or i['news_desk'] == 'Metro'and i["section_name"] != "Great Homes & Destinations":
                count +=1
                dic['id'] = i['_id']
                if i['abstract'] is not None:
                    dic['abstract'] = i['abstract'].encode("utf8")
                dic['headline'] = i['headline']['main'].encode("utf8")
                dic['desk'] = i['news_desk']
                dic['date'] = i['pub_date'][0:10] # cutting time of day.
                dic['section'] = i['section_name']
                if i['snippet'] is not None:
                    dic['snippet'] = i['snippet'].encode("utf8")
                dic['source'] = i['source']
                dic['url'] = i['web_url']
                dic['word_count'] = i['word_count']
                dic['neighborhood'] = name_hood
                dic['year'] = year
                dic['borough'] = "Brooklyn"
                dic['count'] = count 
            else:
                continue
            print dic
            #nb.save(dic)

2010 Bay Ridge
********************
{'count': 1, 'neighborhood': 'Bay Ridge', 'headline': 'A Veteran Goes House-Hunting on the G.I. Bill', 'section': u'Real Estate', 'year': '2010', 'word_count': u'1279', 'snippet': 'Willie Holmes, a young veteran, knew exactly what he wanted in a home. His housing allowance from the G.I. Bill was adequate, but landlords seemed unimpressed.', 'source': u'The New York Times', 'url': u'http://www.nytimes.com/2010/05/30/realestate/30hunt.html', 'desk': u'RealEstate', 'date': u'2010-05-30', 'borough': 'Brooklyn', 'id': u'543f6a6938f0d853e89f6eeb'}
{'count': 2, 'neighborhood': 'Bay Ridge', 'headline': 'Yikes! It\xe2\x80\x99s a Garage', 'section': u'Real Estate', 'year': '2010', 'word_count': u'2146', 'snippet': 'The building of few private garages has set off municipal alarm bells. Fearing copycats, the city has moved swiftly to tighten the rules.', 'source': u'The New York Times', 'url': u'http://www.nytimes.com/2010/08/01/realestate/01cov.html', 'desk': u

In [165]:
meta_dict = {} 
for f in files:
    year = f[-10:-6]
    name_hood = f[1:-10]
    f_path ='/Users/gfsoileau/Metis/Fletcher/times_data/'+f
    with open(f_path) as json_file:
        data = json_file.read().replace('\n', '')
        times_dict = json.loads(data)
        for k, v in times_dict['response']['meta'].items():
            meta_dict[k]=v
            
print meta_dict


{u'hits': 16, u'offset': 0, u'time': 75}


In [136]:
db = conn.ny_times
conn.database_names()

[u'dsbc',
 u'local',
 u'metal',
 u'mydb',
 u'ny_times',
 u'tweet_results',
 u'twitter_results']

In [161]:
db.collection_names()

[u'system.indexes']

In [162]:
conn.ny_times.drop_collection('neighborhood_data')

In [163]:
nb = db.neighborhood_data

In [184]:
list(nb.find())[1000:1010]

[{u'_id': ObjectId('556c9dc35182c3f8f9afdb30'),
  u'borough': u'Brooklyn',
  u'count': 2,
  u'date': u'2011-08-09',
  u'desk': u'Metro',
  u'headline': u'Before Taking on Albany, Raising 14 Children in 1960s Brooklyn',
  u'id': u'54112d4438f0d80f29522361',
  u'neighborhood': u'Park Slope',
  u'section': u'N.Y. / Region',
  u'snippet': u'Before he became governor in 1975, Hugh L. Carey and his wife raised 14 children in a three-story brick house in Park Slope.',
  u'source': u'The New York Times',
  u'url': u'http://www.nytimes.com/2011/08/09/nyregion/before-becoming-governor-carey-raised-14-kids-in-brooklyn.html',
  u'word_count': u'871',
  u'year': u'2011'},
 {u'_id': ObjectId('556c9dc35182c3f8f9afdb31'),
  u'borough': u'Brooklyn',
  u'count': 3,
  u'date': u'2011-02-17',
  u'desk': u'RealEstate',
  u'headline': u'What You Get for ... $800,000',
  u'id': u'540acf3b38f0d84fb88340fe',
  u'neighborhood': u'Park Slope',
  u'section': u'Great Homes & Destinations',
  u'snippet': u'A four-b

In [251]:
cursor = nb.find({"neighborhood":"Park Slope", 'year':'2010'})

In [168]:
hood_names = ['Bay_Ridge','Bedford_Stuyvesant','Bensonhurst','Bergen_Beach','Boerum_Hill','Borough_Park','Brighton_Beach',
           'Brooklyn_Heights','Brooklyn_Navy_Yard','Brownsville','Bushwick','Canarsie','Carroll_Gardens',
           'Clinton_Hill','Cobble_Hill','Coney_Island','Crown_Heights',
           'Cypress_Hills','Ditmas_Park','Downtown_Brooklyn','DUMBO','Dyker_Heights','East_New_York',
           'East_Williamsburg','Farragut','Flatbush','Flatlands','Fort_Greene','Fort_Hamilton','Gerritsen_Beach',
           'Gowanus','Gravesend','Greenpoint','Greenwood_Heights','Kensington',
           'Manhattan_Beach','Marine_Park','Midwood','Mill_Basin','New_Lots','New_Utrecht',
           'Ocean_Parkway','Old_Mill_Basin','Park_Slope','Prospect_Heights',
           'Prospect_Lefferts_Gardens','Prospect_Park_South','Red_Hook','Seagate','Sheepshead_Bay',
           'Stuyvesant_Heights','Sunset_Park','Vinegar_Hill','White_Sands','Williamsburg','Windsor_Terrace']

1286

In [294]:
hood_sample = ['Bay Ridge'] #,'Bedford Stuyvesant','Bensonhurst']
year_sample = ['2014','2013','2012']


num_articles_all = []
#num_articles_year = 0

for h in hood_sample:
    
    for y in year_sample:
        num_articles_year = 0
        cursor = nb.find({'neighborhood':h, 'year':y}, {'count':1,'_id':False})
        for i in cursor:
            num_articles_year = i
        num_articles_all.append(num_articles_year)
print num_articles_all

[{u'count': 9}, {u'count': 9}, {u'count': 4}]


In [264]:
print num_articles_all

[0, 0, 0]


In [283]:
cursor = nb.find({'neighborhood':'Park Slope', 'year':'2010'}, {'count':1,'_id':False})

num_articles_all = []
num_articles_year = 0
for i in cursor:
    num_articles_year = i

num_articles_all.append(num_articles_year)
print num_articles_year

{u'count': 7}


In [262]:
print num_articles_all

[{u'count': 7}]
